In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
from tqdm import tqdm 
import os
## Change directory to your working directory
os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\Amazon Employee Access\Dataset_10')

In [2]:
df=pd.read_csv('Amazon_employee_access.csv')
df.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,target
0,39353,85475,117961,118300,123472,117905,117906,290919,117908,1
1,17183,1540,117961,118343,123125,118536,118536,308574,118539,1
2,36724,14457,118219,118220,117884,117879,267952,19721,117880,1
3,36135,5396,117961,118343,119993,118321,240983,290919,118322,1
4,42680,5905,117929,117930,119569,119323,123932,19793,119325,1


In [3]:
df1=df.astype(str)
df1['target']=df1['target'].astype(int)

In [4]:
df1=df1.drop(['ROLE_TITLE','ROLE_FAMILY'],axis=1)


In [5]:
df1['target']=df1['target'].map({1:0,0:1})

In [6]:
df1['ROLLUP_12']=df1['ROLE_ROLLUP_1']+df1['ROLE_ROLLUP_2']
df1=df1.drop(['ROLE_ROLLUP_1','ROLE_ROLLUP_2'],axis=1)

In [7]:
df1.head()

,RESOURCE,MGR_ID,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target,ROLLUP_12
0,39353,85475,123472,117906,117908,0,117961118300
1,17183,1540,123125,118536,118539,0,117961118343
2,36724,14457,117884,267952,117880,0,118219118220
3,36135,5396,119993,240983,118322,0,117961118343
4,42680,5905,119569,123932,119325,0,117929117930


In [8]:
def create_feat(df,cols):
    cols=list(cols)
    t=df.copy()
    #v=xts.copy()
    #cols.remove(target)
    for col in cols:
        c=cols.copy()
        c.remove(col)
        for i in c:
            d=df.groupby(col)[i].agg('nunique')
            #d1=xts.groupby(col)[i].agg('nunique')
            t[i+'/'+col]=t[col].map(d)
            #v[i+'/'+col]=v[col].map(d1)
            #v[i+'/'+col]=v[col].map(d)
    return t

In [9]:
def aucroc(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    return auc
    #print('AUC_ROC= ',auc)
    #print('Accuracy= ',acc)
    #print('confusion matrix:\n',confusion_matrix(y,pred))

In [44]:
def eliminate(algo,cols,xtr,ytr,xts,yts):
    mod=algo.fit(xtr[cols],ytr)
    auc=aucroc(mod,xts[cols],yts)
    
    high=0
    #removed=[]
    for i in tqdm(cols):
        c=cols.copy()
        c.remove(i)
        mod=algo.fit(xtr[c],ytr)
        auc_sc=aucroc(mod,xts[c],yts)
        if auc_sc>high:
            high=auc_sc
            r=i
        
    return high,r

In [60]:
def col_select(algo,step):
    
    
    cols=list(xtr.columns)
    p=[]
    removed=[]
    mod=algo.fit(xtr[cols],ytr)
    auc=aucroc(mod,xts[cols],yts)
    high_current=auc
    removed_new=[]
    high_new=auc
    best=auc
   # best_rem=[]
    while high_current-high_new<step:    


        removed=removed+removed_new
        print(len(removed))
        removed_new=[]


        if high_current==auc:
            pass
        else:
            high_current=high_new
        if len(cols)>2:
            high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
            while high_new>=high_current:
                cols.remove(r)
                removed_new.append(r)
                high_current=high_new

                high_new,r=eliminate(algo,cols,xtr,ytr,xts,yts)
                print(high_new)
                if high_new>best:
                    best=high_new
                    best_rem=removed+removed_new+[r]
        else:
            break

    print('high_auc=',high_current)
    print('auc_one step ahead=',high_new)
    print('removed: ',removed)
    print('len=',len(removed))
    print('best=',best)
    print('best_rem=',best_rem)

In [12]:
def AST_ecoding(train,test,columns,target,weight):
    t=train.copy()
    v=test.copy()
    gm=t[target].mean()
    for col in columns:
        gb=t.groupby(col)[target].agg(['count','mean'])
        count=gb['count']
        mean=gb['mean']
        gb['smooth']=(count*mean + weight*gm)/(count+weight)
        t[col+'_enc']=t[col].map(gb['smooth'])
        
        v[col+'_enc']=v[col].map(gb['smooth'])
        v[col+'_enc']=v[col+'_enc'].fillna(gm)
    v=v.drop(columns,axis=1)
    t=t.drop(columns,axis=1)
    return t,v

In [13]:
from lightgbm import LGBMClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
lgb=LGBMClassifier(n_estimators=200)
lr=LogisticRegression()

In [14]:

columns=list(df1.columns)
columns.remove('target')
d=create_feat(df1,columns)
t,v=train_test_split(d,test_size=0.3,random_state=123)

target='target'


In [15]:
t.head()

,RESOURCE,MGR_ID,ROLE_DEPTNAME,ROLE_FAMILY_DESC,ROLE_CODE,target,ROLLUP_12,MGR_ID/RESOURCE,ROLE_DEPTNAME/RESOURCE,ROLE_FAMILY_DESC/RESOURCE,...,RESOURCE/ROLE_CODE,MGR_ID/ROLE_CODE,ROLE_DEPTNAME/ROLE_CODE,ROLE_FAMILY_DESC/ROLE_CODE,ROLLUP_12/ROLE_CODE,RESOURCE/ROLLUP_12,MGR_ID/ROLLUP_12,ROLE_DEPTNAME/ROLLUP_12,ROLE_FAMILY_DESC/ROLLUP_12,ROLE_CODE/ROLLUP_12
7693,25285,51761,120370,117906,117908,0,117961118413,2,1,1,...,1938,592,163,49,39,611,93,23,83,40
17676,79168,25396,117941,117913,117888,0,118602118603,12,3,6,...,197,434,13,21,62,55,27,5,20,9
1969,33329,8028,124948,130134,120954,0,117902117903,8,3,8,...,107,23,16,9,9,319,82,37,49,34
15050,3690,3692,126229,207709,121596,0,117961118413,1,1,1,...,124,16,18,8,10,611,93,23,83,40
345,77203,15621,120943,126684,126685,1,117961119256,16,2,6,...,43,34,5,11,6,46,28,5,14,8


In [16]:
#v.isnull().sum()

In [17]:
org_feat=['RESOURCE','MGR_ID','ROLE_DEPTNAME','ROLE_FAMILY_DESC','ROLE_CODE','ROLLUP_12']
t,v=AST_ecoding(t,v,org_feat,target,20)
t.head()

,target,MGR_ID/RESOURCE,ROLE_DEPTNAME/RESOURCE,ROLE_FAMILY_DESC/RESOURCE,ROLE_CODE/RESOURCE,ROLLUP_12/RESOURCE,RESOURCE/MGR_ID,ROLE_DEPTNAME/MGR_ID,ROLE_FAMILY_DESC/MGR_ID,ROLE_CODE/MGR_ID,...,MGR_ID/ROLLUP_12,ROLE_DEPTNAME/ROLLUP_12,ROLE_FAMILY_DESC/ROLLUP_12,ROLE_CODE/ROLLUP_12,RESOURCE_enc,MGR_ID_enc,ROLE_DEPTNAME_enc,ROLE_FAMILY_DESC_enc,ROLE_CODE_enc,ROLLUP_12_enc
7693,0,2,1,1,1,1,15,1,1,2,...,93,23,83,40,0.052553,0.156247,0.038472,0.065241,0.035277,0.028338
17676,0,12,3,6,3,1,8,2,7,4,...,27,5,20,9,0.129880,0.039868,0.076784,0.033729,0.036288,0.120699
1969,0,8,3,8,6,4,4,1,3,3,...,82,37,49,34,0.038539,0.046246,0.101101,0.038227,0.010142,0.036647
15050,0,1,1,1,1,1,33,3,6,6,...,93,23,83,40,0.055055,0.023123,0.033690,0.037295,0.130292,0.028338
345,1,16,2,6,4,1,3,1,2,2,...,28,5,14,8,0.092828,0.093746,0.070443,0.148434,0.182512,0.108729


In [42]:

xtr=t.drop('target',axis=1)
xts=v.drop(target,axis=1)
ytr=t[target]
yts=v[target]
cols=list(xtr.columns)


In [45]:
col_select(lgb,0.05)

0


100%|██████████| 35/35 [00:59<00:00,  1.71s/it]


0.8387537683500207


100%|██████████| 34/34 [00:49<00:00,  1.47s/it]


0.8438954620933301


...

0.8266060641593506
32


100%|██████████| 3/3 [00:02<00:00,  1.43it/s]


0.7934042054218851
33


100%|██████████| 2/2 [00:01<00:00,  1.33it/s]


0.6725502415148293
high_auc= 0.7934042054218851
auc_one step ahead= 0.6725502415148293
removed:  ['RESOURCE_enc', 'MGR_ID_enc', 'ROLLUP_12_enc', 'ROLE_FAMILY_DESC_enc', 'ROLE_CODE_enc', 'ROLE_DEPTNAME_enc', 'ROLLUP_12/RESOURCE', 'MGR_ID/ROLE_CODE', 'ROLE_DEPTNAME/MGR_ID', 'ROLE_CODE/ROLE_FAMILY_DESC', 'RESOURCE/ROLE_CODE', 'ROLE_CODE/ROLE_DEPTNAME', 'ROLE_FAMILY_DESC/ROLE_CODE', 'MGR_ID/ROLLUP_12', 'RESOURCE/ROLE_FAMILY_DESC', 'ROLE_FAMILY_DESC/ROLLUP_12', 'ROLE_CODE/MGR_ID', 'ROLE_FAMILY_DESC/ROLE_DEPTNAME', 'ROLE_DEPTNAME/RESOURCE', 'ROLE_CODE/ROLLUP_12', 'RESOURCE/ROLLUP_12', 'ROLE_DEPTNAME/ROLLUP_12', 'ROLE_FAMILY_DESC/RESOURCE', 'ROLLUP_12/ROLE_DEPTNAME', 'MGR_ID/ROLE_FAMILY_DESC', 'ROLE_DEPTNAME/ROLE_CODE', 'ROLLUP_12/MGR_ID', 'MGR_ID/RESOURCE', 'MGR_ID/ROLE_DEPTNAME', 'ROLLUP_12/ROLE_FAMILY_DESC', 'ROLE_FAMILY_DESC/MGR_ID', 'ROLE_CODE/RESOURCE', 'ROLE_DEPTNAME/ROLE_FAMILY_DESC']
len= 33
best= 0.8687810404239464
best_rem= ['RESOURCE_enc', 'MGR_ID_enc', 'ROLLUP_12_enc', 'ROLE_FAMI

In [52]:
rem=['RESOURCE_enc', 'MGR_ID_enc', 'ROLLUP_12_enc', 'ROLE_FAMILY_DESC_enc', 'ROLE_CODE_enc', 'ROLE_DEPTNAME_enc', 
     'ROLLUP_12/RESOURCE','MGR_ID/ROLE_CODE']
t1=t.drop(rem,axis=1)
v1=v.drop(rem,axis=1)

In [53]:
def aucroc1(mod,x,y):
    pred=mod.predict(x)
    prob=mod.predict_proba(x)[:,1]
    auc=roc_auc_score(y,prob)
    acc=accuracy_score(y,pred)
    #return auc
    print('AUC_ROC= ',auc)
    print('Accuracy= ',acc)
    print('confusion matrix:\n',confusion_matrix(y,pred))

In [54]:
xtr=t1.drop('target',axis=1)
xts=v1.drop(target,axis=1)
ytr=t1[target]
yts=v1[target]
cols=list(xtr.columns)
mod=lgb.fit(xtr,ytr)
aucroc1(mod,xts,yts)

AUC_ROC=  0.8687810404239464
Accuracy=  0.9501576645305666
confusion matrix:
 [[9205   55]
 [ 435  136]]


In [58]:
prob=mod.predict_proba(xts)[:,1]
pred=mod.predict(xts)
res=pd.DataFrame({'Prob_LGB':prob,'Pred_LGB':pred})
res.to_csv('result_LGB.csv')

In [ ]:
xtr=t.drop('target',axis=1)
xts=v.drop(target,axis=1)
ytr=t[target]
yts=v[target]
cols=list(xtr.columns)
lr=LogisticRegression(max_iter=10e8)
col_select(lr,0.05)